In [34]:
from selenium import webdriver
import requests
import time
import os
from pathlib import Path
import pandas as pd
from datetime import datetime

import pathlib
import shutil

def ontem(hoje):
    distD = int(hoje.strftime('%d'))-1
    distM = int(hoje.strftime('%m'))
    distA = 21
    if distD < 10:
        return '0{}/0{}/{}'.format(str(distD),str(distM),str(distA))
    return '{}/0{}/{}'.format(str(distD),str(distM),str(distA))

def dia_de_anteontem(hoje):
    dist = int(hoje.strftime('%d'))-2
    if dist < 10:
        return 'O'+str(dist)
    else:
        return str(dist)

def dia_de_ontem(hoje):
    dist = int(hoje.strftime('%d'))-1
    if dist < 10:
        return 'O'+str(dist)
    else:
        return str(dist)


def download_wait(directory, timeout, nfiles=None):
    seconds = 0
    dl_wait = True
    while dl_wait and seconds < timeout:
        time.sleep(1)
        dl_wait = False
        files = os.listdir(directory)
        if nfiles and len(files) != nfiles:
            dl_wait = True

        for fname in files:
            if fname.endswith('.crdownload'):
                dl_wait = True

        seconds += 1
    return seconds

def download_check(directory, file_name):
    seconds = 0
    no_dl = True
    files = os.listdir(directory)
    for fname in files:
        if fname.endswith('{}'.format(file_name)):
            return True
    raise ValueError("Nao está")

lista=['isolamento','leitos_ocupados_por_unidade_hospitalar','vacinometro','dados_covid_municipios_sp']
home = Path.home()
parent = pathlib.Path().absolute()
d = Path(home,'Downloads')
cidade='piracicaba'
t=Path(parent,"bases/{}/".format(cidade))
source_dir = d
target_dir = t
here = pathlib.Path().absolute()
path_win = Path(here,"chromedriver_ln")


hoje = datetime.now()
dhoje = hoje.strftime('%d')
dontem = dia_de_ontem(hoje)
danteontem = dia_de_anteontem(hoje)
mes= hoje.strftime('%m')
hora = hoje.strftime('%H')
noite=False
for item in lista:
    driver = webdriver.Chrome(path_win)
    path_download = Path(parent,"bases/{}/{}.csv".format(cidade,item))
    
    link1 = "https://www.saopaulo.sp.gov.br/wp-content/uploads/2021/{}/2021{}{}_{}.csv".format(mes,mes,dhoje,item)
    link2= "https://www.saopaulo.sp.gov.br/wp-content/uploads/2021/{}/2021{}{}_{}.csv".format(mes,mes,dontem,item)
        

    t1=Path(parent,"bases/{}/2021{}{}_{}.csv".format(cidade,mes,dhoje,item))
    t2=Path(parent,"bases/{}/2021{}{}_{}.csv".format(cidade,mes,dontem,item))
    t3=Path(parent,"bases/{}/2021{}{}_{}.csv".format(cidade,mes,danteontem,item))

    d1=Path(d,"2021{}{}_{}.csv".format(mes,dhoje,item))
    d11=Path(d,"2021{}{}_{} (1).csv".format(mes,dhoje,item))
    d2=Path(d,"2021{}{}_{}.csv".format(mes,dontem,item))
    d21=Path(d,"2021{}{}_{} (1).csv".format(mes,dontem,item))
    d3=Path(d,"2021{}{}_{}.csv".format(mes,danteontem,item))
    d31=Path(d,"2021{}{}_{} (1).csv".format(mes,danteontem,item))


    arq1 = "2021{}{}_{}.csv".format(mes,dhoje,item)
    arq2 = "2021{}{}_{}.csv".format(mes,dontem,item)
    arq3 = "2021{}{}_{}.csv".format(mes,danteontem,item)
    
    try:
        noite=True
        data = hoje.strftime('%d/%m/%y')
        if (item == 'dados_covid_municipios_sp'):
            driver.get(link1)
            driver.implicitly_wait(10)
            download_wait(d,190)
            download_check(d,'dados_covid_municipios_sp.csv')
            driver.quit()
        elif (item == 'isolamento'):
            driver.get(link1)
            driver.implicitly_wait(10)
            download_wait(d,130)
            download_check(d,'isolamento.csv')
            driver.quit()
        elif (item == 'vacinometro'):
            driver.get(link1)
            driver.implicitly_wait(10)
            download_wait(d,130)
            download_check(d,'vacinometro.csv')
            driver.quit()
        elif (item == 'leitos_ocupados_por_unidade_hospitalar'):
            driver.get(link1)
            driver.implicitly_wait(10)
            download_wait(d,130)
            download_check(d,'leitos_ocupados_por_unidade_hospitalar.csv')
            driver.quit()
        else:
            raise ValueError("Nao está")

    except:
        noite=False
        data = ontem(hoje)
        if (item == 'dados_covid_municipios_sp'):
            driver.get(link2)
            driver.implicitly_wait(10)
            download_wait(d,190)
            driver.quit()
        elif (item == 'isolamento'):
            driver.get(link2)
            driver.implicitly_wait(10)
            download_wait(d,130)
            driver.quit()
        else:
            driver.get(link2)
            driver.implicitly_wait(10)
            download_wait(d,30)
            driver.quit()
    

    if path_download.is_file():
        vacinas=pd.read_csv(path_download,',')
    else:
        vacinas=pd.DataFrame()
    file_names = os.listdir(source_dir)
    for file_name in file_names:
        if noite:
            if (file_name==arq1) and not(t1.is_file()):
                shutil.move(os.path.join(source_dir, file_name), target_dir)
        else:
            if file_name==arq2 and not(t2.is_file()):
                shutil.move(os.path.join(source_dir, file_name), target_dir)
            continue

    if (t2.is_file())&(t1.is_file())& noite :
        os.remove(t2)

    elif (t3.is_file()) & (t2.is_file()):
        os.remove(t3)

    if d1.is_file():
        os.remove(d1)
    if d2.is_file():
        os.remove(d2)
    if d3.is_file():
        os.remove(d3)
    if d11.is_file():
        os.remove(d11)
    if d21.is_file():
        os.remove(d21)
    if d31.is_file():
        os.remove(d31)
        
    if noite:
        try:
            df = pd.read_csv(t1,';')
        except FileNotFoundError:
            df = pd.read_csv(t2,';')
    else:
        df = pd.read_csv(t2,';')
    if item=='leitos_ocupados_por_unidade_hospitalar':
        try:
            df = df[df['Município'] == 'PIRACICABA']
            df['Data'] = data
        except:
            df = df[df.iloc[:,1] == 'PIRACICABA']
            df['Data'] = data
    elif item=='isolamento':
        try:
            df = df[df['Município1'] == 'PIRACICABA']
        except:
            df = df = df[df.iloc[:,0] == 'PIRACICABA']    
    elif item=='dados_covid_municipios_sp':
        try:
            df = df[df['nome_munic'] == 'PIRACICABA']
        except:
            df = df = df[df.iloc[:,0] == 'PIRACICABA']   
    elif item=='vacinometro':
        try:
            df = df[df['Municipio'] == 'PIRACICABA']
            df['Data'] = data
        except:
            df = df = df[df.iloc[:,0] == 'PIRACICABA']        
    else:
        continue
        
    if (path_download.is_file()):
        if vacinas['Data'].iloc[0]!= data:
            vacinas = vacinas.append(df,ignore_index=True)
    else:
        vacinas = vacinas.append(df,ignore_index=True)
    vacinas.to_csv(path_download,index=False)


27

In [6]:
from selenium import webdriver
import requests
import time
import os
from pathlib import Path
import pandas as pd
from datetime import datetime

import pathlib
import shutil

# -----------------------------funções-------------------------------------
def ontem(hoje):
    distD = int(hoje.strftime('%d'))-1
    distM = int(hoje.strftime('%m'))-1
    distA = 2021
    if dist < 10:
        return 'O'+str(dist)
    else:
        return str(dist)
    
def dia_de_anteontem(hoje):
    dist = int(hoje.strftime('%d'))-2
    if dist < 10:
        return 'O'+str(dist)
    else:
        return str(dist)

def dia_de_ontem(hoje):
    dist = int(hoje.strftime('%d'))-1
    if dist < 10:
        return 'O'+str(dist)
    else:
        return str(dist)


def download_wait(path_to_downloads):
    seconds = 0
    dl_wait = True
    while dl_wait and seconds < 10:
        time.sleep(0.2)
        dl_wait = False
        for fname in os.listdir(path_to_downloads):
            if fname.endswith('.xlsx'):
                dl_wait = True
        seconds += 1
    return seconds

# ---------------definincdo os paths p/ downloads e para DB do projeto--
home = Path.home()
parent = pathlib.Path().absolute()
cidade='piracicaba'
path_download = Path(parent,"bases/{}/vacinometro.csv".format(cidade))
d = Path(home,'Downloads')
t=Path(parent,"bases/{}/".format(cidade))
source_dir = d
target_dir = t

here = pathlib.Path().absolute()

path_win = Path(here,"chromedriver_ln")

# ----------------------iniciando o driver---
driver = webdriver.Chrome(path_win)


hoje = datetime.now()

dhoje = hoje.strftime('%d')
dontem = dia_de_ontem(hoje)
danteontem = dia_de_anteontem(hoje)
mes= hoje.strftime('%m')
hora = hoje.strftime('%H')
noite=False

if int(hora)<=17:
    link= "https://www.saopaulo.sp.gov.br/wp-content/uploads/2021/{}/2021{}{}_isolamento.csv".format(mes,mes,dontem)
    data = ontem(hoje)
else:
    noite=True
    data = hoje.strftime('%d/%m/%y')
    link = "https://www.saopaulo.sp.gov.br/wp-content/uploads/2021/{}/2021{}{}_isolamento.csv".format(mes,mes,dhoje)





t1=Path(parent,"bases/{}/2021{}{}_isolamento.csv".format(cidade,mes,dhoje))
t2=Path(parent,"bases/{}/2021{}{}_isolamento.csv".format(cidade,mes,dontem))
t3=Path(parent,"bases/{}/2021{}{}_isolamento.csv".format(cidade,mes,danteontem))

d1=Path(d,"2021{}{}_isolamento.csv".format(mes,dhoje))
d11=Path(d,"2021{}{}_isolamento(1).csv".format(mes,dhoje))
d2=Path(d,"2021{}{}_isolamento.csv".format(mes,dontem))
d21=Path(d,"2021{}{}_isolamento(1).csv".format(mes,dontem))
d3=Path(d,"2021{}{}_isolamento.csv".format(mes,danteontem))
d31=Path(d,"2021{}{}_isolamento(1).csv".format(mes,danteontem))


arq1 = "2021{}{}_isolamento.csv".format(mes,dhoje)
arq2 = "2021{}{}_isolamento.csv".format(mes,dontem)
arq3 = "2021{}{}_isolamento.csv".format(mes,danteontem)


driver.get(link)
driver.implicitly_wait(10)
download_wait(d)
driver.quit()


vacinas=pd.read_csv(path_download,',')
file_names = os.listdir(source_dir)
for file_name in file_names:
    if noite:
        if (file_name==arq1) and not(t1.is_file()):
            shutil.move(os.path.join(source_dir, file_name), target_dir)
    else:
        if file_name==arq2:
            shutil.move(os.path.join(source_dir, file_name), target_dir)
        continue

if (t2.is_file())&(t1.is_file())& noite :
    os.remove(t2)

elif (t3.is_file()) & (t2.is_file()):
    os.remove(t3)

if d1.is_file():
    os.remove(d1)
if d2.is_file():
    os.remove(d2)
if d3.is_file():
    os.remove(d3)
if d11.is_file():
    os.remove(d11)
if d21.is_file():
    os.remove(d21)
if d31.is_file():
    os.remove(d31)
if noite:
    df = pd.read_csv(t1,';')
else:
    df = pd.read_csv(t2,';')
df = df[df['Municipio'] == 'PIRACICABA']
df['Data'] = data
if vacinas['Data'].iloc[0]!= data:
    vacinas = vacinas.append(df,ignore_index=True)
vacinas.to_csv(path_download,index=False)


FileNotFoundError: [Errno 2] No such file or directory: '/home/yan/projetos/curiocidades/notebook/bases/piracicaba/20210228_isolamento.csv'

In [5]:
int(hora)

18

In [ ]:
cidade='piracicaba'
path_download15 = Path(parent,"bases/{}/covid.csv".format(cidade))

hoje = datetime.now()
dhoje = hoje.strftime('%d')
mes= hoje.strftime('%m')

covid = pd.read_csv(path_download15)
covid.drop('Unnamed: 0',axis=1,inplace=True)
here = pathlib.Path().absolute()
path_win = Path(here,"chromedriver_ln")
driver = webdriver.Chrome(path_win)

dias = ['{}'.format(dhoje)]
meses =['{}'.format(mes)]
for m in meses:
    for d in dias:
        link = 'http://www.piracicaba.sp.gov.br/plantao+coronavirus+{}+{}+2021.aspx'.format(d,m)
        print(link)
        driver.get(link)
        div = driver.find_element_by_id('imagenet-conteudo')
        novos_obitos = 'NA'
        casos_confirmados = 'NA'
        casos_suspeitos = 'NA'
        casos_descartados = 'NA'
        casos_recuperados = 'NA'
        em_tratamento = 'NA'
        obitos = 'NA'
        homens = 'NA'
        mulheres = 'NA'
        for i in range(len(div.text.split('\n'))):
            if (len(div.text.split('\n')[i].split())<5)&(len(div.text.split('\n')[i].split())>1):
                if div.text.split('\n')[i].split()[-1] =='confirmados':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        casos_confirmados = div.text.split('\n')[i].split()[0][:6]
                    else:
                        casos_confirmados = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='suspeitos':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        casos_suspeitos = div.text.split('\n')[i].split()[0][:6]
                    else:
                        casos_suspeitos = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='descartados':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        casos_descartados = div.text.split('\n')[i].split()[0][:6]
                    else:
                        casos_descartados = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='recuperados':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        casos_recuperados = div.text.split('\n')[i].split()[0][:6]
                    else:
                        casos_recuperados = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='tratamento':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        em_tratamento = div.text.split('\n')[i].split()[0][:6]
                    else:
                        em_tratamento = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='óbitos':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        obitos = div.text.split('\n')[i].split()[0][:3]
                    else:
                        obitos = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='óbito':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        obitos = div.text.split('\n')[i].split()[0][:3]
                    else:
                        obitos = div.text.split('\n')[i].split()[0]
            elif len(div.text.split('\n')[i].split())>1:
                if div.text.split('\n')[i].split()[1] =='homens:':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='Homens:':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='homem:':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='Homens':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='homens':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='homem:':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='mulheres:':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='Mulheres:':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='mulher:':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='mulheres':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='Mulheres':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='mulher':
                    mulheres = div.text.split('\n')[i].split()[0]
        dicionario ={'Total Casos Confirmados': int(casos_confirmados.replace('.','')),
                     'Total Casos Suspeitos':int(casos_suspeitos.replace('.','')),
                    'Total Casos Descartados':int(casos_descartados.replace('.','')),
                    'Total Casos Recuperados':int(casos_recuperados.replace('.','')),
                    'Total Casos em Tratamento':int(em_tratamento.replace('.','')),
                     'Total Óbitos':int(obitos.replace('.','')),
                     'Homens':int(homens.replace('.','')),
                     'Mulheres':int(mulheres.replace('.','')),'Data':'{}/{}/2021'.format(d,m)}
        covid = covid.append(dicionario, ignore_index=True)
        covid['Óbitos Diários'].iloc[-1]=covid['Total Óbitos'].iloc[-1]-covid['Total Óbitos'].iloc[-2]
        covid['Casos Diários'].iloc[-1]=covid['Total Casos Confirmados'].iloc[-1]-covid['Total Casos Confirmados'].iloc[-2]
driver.quit()
